Load the search data and do some initial exploration

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

In [ ]:
from tensorflow.python.framework import dtypes

In [ ]:
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
tfclient = BigQueryClient()

In [ ]:
GCP_PROJECT_ID = 'dataorg-hackweek-2021'
DATASET_PROJECT_ID = "moz-fx-data-shared-prod"
DATASET_ID = f"{DATASET_PROJECT_ID}.mlhackweek_search_live"
TABLE_ID = f"{DATASET_ID}.action_v1"

In [ ]:
%load_ext google.cloud.bigquery

Try to load a few columns into TensorFlow. Fails on nested struct columns

In [ ]:
rs = tfclient.read_session(
    f"projects/{GCP_PROJECT_ID}",
    DATASET_PROJECT_ID,
    "action_v1",
    "mlhackweek_search_live",
    ["document_id", "ping_info.end_time"],
    [dtypes.string, dtypes.string],
    "submission_timestamp > '2021-01-19'"
)

In [ ]:
d = rs.parallel_read_rows()

In [ ]:
d

In [ ]:
# This causes the kernel to crash
# list(d.take(5).as_numpy_iterator())

Load the table from BigQuery

In [ ]:
TABLE_ID

In [ ]:
table = client.get_table(TABLE_ID)

Take a look at the schema for the `metrics` columns.

In [ ]:
# Write full schema to JSON file
client.schema_to_json(table.schema, "search_table_schema.json")

Extract some basic info for metrics columns

In [ ]:
# Schema object is nested lists of lists of Field objects
schema_metrics = [x for x in table.schema if x.name == "metrics"][0]

metrics_info = []

for f in schema_metrics.fields:
    for ff in f.fields:
        metrics_info.append({
            "name": f"{f.name}.{ff.name}",
            "type": ff.field_type,
            "description": ff.description.strip() if ff.description else None
        })

In [ ]:
pd.DataFrame(metrics_info)

In [ ]:
%%bigquery

SELECT
    metrics.*
FROM `moz-fx-data-shared-prod.mlhackweek_search_live.action_v1`
where
    DATE(submission_timestamp) >= DATE('2021-01-19')
    AND metrics.string.search_meta_session_id is not NULL
LIMIT 1

Flatten out the nested columns of interest using BigQuery SQL.
The nested columns are arrays of key-value pairs. Corresponding entries match up by position.
To extract nested values, zip together individual arrays into a big array of structs, then unnest that single array, extract values.

In [ ]:
%%bigquery search_data_flat

with subset as (
    SELECT
        metrics.string.search_meta_search_text as search_text,
        metrics.datetime.search_meta_url_select_timestamp as ts,
        metrics.string.search_meta_session_id as session_id,
        metrics.labeled_counter.search_meta_position as position,
        metrics.labeled_string.search_meta_url as url,
        metrics.labeled_string.search_meta_hostname as hostname,
        metrics.labeled_string.search_meta_title as title,
        metrics.labeled_string.search_meta_short_description as descr,
        metrics.labeled_string.search_meta_preamble as preamble,
        metrics.labeled_boolean.search_meta_selected as selected
    FROM
        `moz-fx-data-shared-prod.mlhackweek_search_live.action_v1`
    WHERE
        DATE(submission_timestamp) >= DATE('2021-01-19')
        and metrics.string.search_meta_session_id is not NULL
),
zipped as (
    # Each of the metrics columns are arrays of key-value pairs
    # Entries match up by position, so we can drop keys and extract values
    SELECT
        search_text,
        ts,
        session_id,
        array_agg(
            struct(
                pos.value as position,
                url[offset(pos_offset)].value as url,
                hostname[offset(pos_offset)].value as hostname,
                title[offset(pos_offset)].value as title,
                descr[offset(pos_offset)].value as descr,
                preamble[offset(pos_offset)].value as preamble,
                selected[offset(pos_offset)].value as selected
            )
        ) as arr,
        sum(cast(selected[offset(pos_offset)].value as int)) as n_selected
    # Unnest one of the nested columns, and use its offset (index)
    # to index all of the others.
    FROM subset, unnest(position) as pos
    with offset as pos_offset
    GROUP BY search_text, ts, session_id
)
select
    search_text,
    ts,
    session_id,
    array_length(zipped.arr) as n_results,
    n_selected,
    arr.position,
    arr.url,
    arr.hostname,
    arr.title,
    arr.descr,
    arr.preamble,
    arr.selected
# Implicit unnesting explodes array column
from zipped, zipped.arr

In [ ]:
search_data_flat.head()

Take a look at search sessions: num selected results, num unique queries, num total queries

In [ ]:
(
    search_data_flat
    .groupby(["session_id", "n_selected"])
    .agg({"search_text": ["nunique", "count"]})
    .rename(columns={"nunique": "n_unique_query_text", "count": "n_queries_issued"})
    .droplevel(0, axis="columns")
    .sort_values(["n_unique_query_text", "n_queries_issued"], ascending=False)
    .reset_index()
)